In [59]:
spark.sql(
    '''
       CREATE TABLE IF NOT EXISTS txkt.dim_organ_df
        (
         `organ_name`           STRING COMMENT '机构名称',
         `organ_praise_degree`  STRING COMMENT '机构好评度',
         `organ_video_num`      STRING COMMENT '机构课程数',
         `organ_all_person_num` STRING COMMENT '机构总学习人次',
         `etl_time`             STRING COMMENT '数据加工时间'      
       ) 
       PARTITIONED BY (`dt` STRING) 
       STORED AS PARQUET
    '''
)


DataFrame[]

In [8]:
spark.sql(
    'INSERT OVERWRITE TABLE txkt.dim_organ_df PARTITION (dt)' + 
    '''
        SELECT organ_name,         
               organ_praise_degree,         
               organ_video_num,                 
               organ_all_person_num,          
               FROM_UNIXTIME(CAST(NOW() AS BIGINT), 'yyyy-MM-dd HH:mm:ss') AS elt_time,
               DATE_FORMAT(TO_TIMESTAMP(grab_time, 'yyyy-MM-dd HH:mm:ss'), 'yyyy-MM-dd') AS dt 
          FROM ( 
                 SELECT organ_name,         
                        organ_praise_degree,         
                        organ_video_num,                 
                        organ_all_person_num,           
                        grab_time,
                        ROW_NUMBER() OVER(PARTITION BY organ_name ORDER BY grab_time DESC) AS rk
                   FROM txkt.ods_tencent_study_video
                  WHERE dt='{dt}' 
               ) AS a
         WHERE rk=1
    '''.format(dt='2022-03-27')
)

DataFrame[]

In [9]:
spark.sql("SELECT dt, COUNT(*) FROM txkt.dim_organ_df GROUP BY dt ORDER BY dt;").show()

+----------+--------+
|        dt|count(1)|
+----------+--------+
|2022-03-13|    7998|
|2022-03-23|    8081|
|2022-03-24|    8072|
|2022-03-25|    8018|
|2022-03-26|    8074|
|2022-03-27|    8051|
+----------+--------+



In [66]:
spark.sql(
    '''
    SELECT organ_name, COUNT(*) AS cnt
      FROM txkt.dim_organ_df
     WHERE dt='2022-03-23'
  GROUP BY organ_name
    HAVING cnt>1
    '''
).show()

+----------+---+
|organ_name|cnt|
+----------+---+
+----------+---+

